<img src = "../../Data/bgsedsc_0.jpg">
$\newcommand{\bb}{\boldsymbol{\beta}}$
$\DeclareMathOperator{\Gau}{\mathcal{N}}$
$\newcommand{\bphi}{\boldsymbol \phi}$
$\newcommand{\bx}{\boldsymbol{x}}$
$\newcommand{\bu}{\boldsymbol{u}}$
$\newcommand{\by}{\boldsymbol{y}}$
$\newcommand{\whbb}{\widehat{\bb}}$
$\newcommand{\hf}{\hat{f}}$
$\newcommand{\tf}{\tilde{f}}$
$\newcommand{\ybar}{\overline{y}}$
$\newcommand{\E}{\mathbb{E}}$
$\newcommand{\Var}{Var}$
$\newcommand{\Cov}{Cov}$
$\newcommand{\Cor}{Cor}$

# High-dimensional predictive regression models - more advanced concepts


In [1]:
# We load the relevant modules

%matplotlib inline
import matplotlib.pylab as plt

import pandas as pd
#seaborn is a module for figures
import seaborn as sns
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression

## The bias-variance tradeoff in Statistics and Machine Learning - little maths behind the picture

The picture tells it all! (Taken from Bishop's book)

<img src="../images/bias_variance_bishop.png" width="400">

### A little math: bias-variance tradeoff

Suppose that the data are generated according to a mechanism: 

$$ y = h(\bx) + \epsilon$$

where 

+ $\bx \sim p(\bx)$ 
+ $\E[\epsilon] = 0$
+ $\Var(\epsilon) = v$
+ $\Cor(\bx,\epsilon) = 0$

Also, let 

$$b_n(\bu) = \E[\hf_n(\bu)]$$ 

$$c_n(\bu) = \Var(\hf_n(\bu))$$

Then: 

$$MSE_n = \E[(y^*-\hf_n(\bx^*))^2] = v + \E[(b_n(\bx^*) - h(\bx^*))^2] + \E[c_n(\bx^*)]$$

+ The first term relates to the prediction problem: our algorithm cannot affect it
+ The other two relate to the algorithm:
     + Algorithms that underfit are those with small third term (small variance) and large second term (high bias)
     + Algorithms that overfit are those with small second term (small bias) and large third term (high variance)
     + For linear model the story is simple and mathematics can back it up: adding features increases the variance, removing features might increase the bias
+ The MSE balances bias and variance: we want **algorithms that can strike a good bias-variance tradeoff**! 


### Model selection criteria

Model selection criteria balance in-sample fit with model complexity

For example, one such criterion is the so-called Akaike Information Criterion (AIC). For linear models this is equivalent to another model selection criterion, the so-called Mallows's $C_p$. 

AIC provides an approximation to a quantity related to but not the same as $MSE_n$ - by doing a bias-correction to the in-sample mean squared error. 

For a statistical model that involves $p$ parameters and loss function which is the log-likelihood, the AIC takes the form: 

$$AIC(\textrm{model}) = -{2 \over n} \textrm{(maximised log-lik of model)} + {2 \over n} p$$

When we deviate from maximum likelihood, the *effective number of parameters* - we will call this **degrees of freedom (df)** differs from $p$. In particular: 

+ Algorithms that have used the response data to select the model, they will have $df \geq p$: effectively more than $p$ parameters have been used to come up with this particular model
+ Algorithms that shrink the parameter values towards zero, they will have $df \leq p$: effectively each parameter counts less - or even 0 if it is shrunk exactly to 0 

Specifically to the case where there is a family of algorithms indexed by a tuning parameter $\lambda$ (e.g. the regularizing parameter) and AIC is used to score each member of the family, the AIC formula becomes: 

$$AIC(\lambda) = -{2 \over n} \textrm{(maximised log-lik of model for given }\lambda) + {2 \over n} df(\lambda)$$

where $df(\lambda)$ are the degrees of freedom for that particular value of the tuning parameter. 

The name of the game is to come up with an estimate of $df(\lambda)$ for the algorithm of interest

### Degrees of freedom of lasso

Rather surprisingly the degrees of freedom for a given value of $\lambda$ used in a lasso algorithm is *unbiasedly estimated* by the resultant number of non-zero coefficients. 

For the sake of clarity, let 
$m_\lambda$ be the size of the model chosen with $\lambda$ regularizer value, and $\lambda_m$ the *transition point* in the lasso path (from empty to full) at which the model size becomes $m$ 

The result for the lasso is that $m_\lambda$ is an unbiased estimator of $df(\lambda)$ - this is established in Zou, Hastie and Tibshirani (2007, Annals of Statistics)

+ Lasso uses the response data to choose the model, hence we would expect $df(\lambda) \geq m_\lambda$ 
+ On the other hand, lasso shrinks coefficients to zero hence we would expect $df(\lambda) \leq m_\lambda$
+ Magically, the two effects cancel out and $m_\lambda$ is unbiased for $df(\lambda)$

Combined with the Gaussian likelihood, the AIC criterion for lasso becomes 

$$AIC(\lambda) = {\sum_i (y_i - \hf(\bx_i))^2 \over n v} + {2 \over n} m_\lambda$$

An important result is that it is only necessary to search among the transition points $\lambda_1,\lambda_2,\ldots$ to identify the model that optimizes the AIC criterion 

## Cross validation

Recall that the ultimate goal in predictive measures of performance is to estimate the mean squared error, 

$$MSE_n = \E[(y^*-\hf_n(\bx^*))^2]$$

Leave-one-out CV provides an estimator of this quantity: recall that:

+ it splits the $n$ available data in $n$ groups - lets call them *folds*
+ using the data in $n-1$ folds as training, it measures squared error on the left-out fold
+ it then averages the squared errors from the $n$ folds

More generally, $K$-fold CV proceeds by: 

+ it splits the $n$ available data in $K$ folds
+ using the data in $K-1$ folds as training, it measures average squared error on the left-out fold
+ it then averages the squared errors from the $K$ folds


Little math shows that the generic formula for the estimator of $MSE_n$ from $K$-fold CV is (for simplicity say $n = Km$, and let $K(i)$ denote the fold the $i$th observation belongs to)

$$
{1 \over n} \sum_i \left ( y_i - \hf_{n-m, -K(i)}(\bx_i) \right)^2 
$$

The bias-variance tradeoff is at work here too:

+ Small $K$, hence large $m$, is good since the estimate of mean squared error in each batch is based on more independent observations, hence *smaller variance*
+ Small $K$, hence $n-m\ll n$, is bad since it means that we are really evaluating $\hf_{n-m}$ which could be significantly different from $\hf_{n}$, hence *larger bias*

For a very simple example, that of evaluating the regression model with intercept only with loss function given by the log-likelihood, we can do the calculations explicitly, and compute the mean squared error of the CV estimator as a function of $K$ - it also depends on second and fourth moments of the data generating process 

<table>
    <tr>
        <td> Gaussian-tailed data </td>
        <td> heavy-tailed data </td>
    </tr>
    <tr>
    <td>  <img src="../images/mse_cv_gauss.png" > </td>
    <td> <img src="../images/mse_cv_student.png"> </td>
    </tr>
</table>

In these examples the minimum is around 3-5 fold, which results in leaving aside 20-30% of the data for testing. 

The story about the optimal choice of $K$ is neither straightforward nor conclusive. Alternative methods, such as the bootstrap, are also competitors to CV for obtaining estimates of prediction error 